### Vehicle Routing Problem

In [2]:
dict_routes = {1: [0,2,5,7,8], 2: [0,3,6,7,8]}

In [3]:
output_text = ''''''
for key in dict_routes.keys():
    output_text += f'''Route {key}: '''
    for value in range(0, len(dict_routes[key])-1):
        output_text += f'''{value} -> '''
    output_text += f'''{dict_routes[key][-1]}\n'''

In [4]:
print(output_text)

Route 1: 0 -> 1 -> 2 -> 3 -> 8
Route 2: 0 -> 1 -> 2 -> 3 -> 8



In [6]:
pip install loguru

  Using cached loguru-0.7.0-py3-none-any.whl (59 kB)
  Using cached win32_setctime-1.1.0-py3-none-any.whl (3.6 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install ortools

     ---------------------------------------- 43.5/43.5 MB 1.7 MB/s eta 0:00:00
     -------------------------------------- 422.5/422.5 kB 2.2 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import numpy as np

import datetime as dt

import json
import logging
import os
import sys

from loguru import logger
import io
import pandas as pd
import calendar
import numpy as np
import plotly.express as px

# functions
import pandas as pd
import calendar
import plotly.graph_objects as go
import datetime as dt

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

sys.path.append('..')

pd.set_option('display.max_columns', None)

from src.distance import get_haversine_distance_lonlat, get_distance_osrm, get_waypoints_osrm

%load_ext autoreload
%autoreload 2

project_path = 's3://sfgdata/projects/sustainable-transport'

ModuleNotFoundError: No module named 's3fs'

In [6]:
def create_distance_matrix(df):
    '''Returns distance in km'''
    x = df[['Receiver longitude', 'Receiver latitude']].to_numpy()
    x = np.vstack((df[['Shipper longitude', 'Shipper latitude']].to_numpy()[0], x))
    x = np.asarray(x)
    m = x.shape[0]
    result = np.empty((m,m),dtype=int)  # FIXME: figure out the best dtype
    for i in range(m):
        for j in range(m):
            result[i,j] = get_haversine_distance_lonlat(x[i],x[j])
            #result[i,j] = get_distance_osrm(x[i],x[j])
    return result

In [7]:
def create_matrix_from_array():

    x = np.zeros([4, 4]) # 4x4 array of zeros

    x[np.triu_indices(4, 1)] = [1, 2, 4, 3, 5, 6]

    x += x.T

In [8]:
def group_by_client_shipment(dff):
    dff = dff.groupby(['Receiver longitude', 'Receiver latitude', "Client name"]).agg({'Sender weight (kg)': 'sum',
                                                                                       'Distance (km)': 'first',
                                                                                        'Shipper latitude': 'first',
                                                                                        'Shipper longitude': 'first'}).reset_index()
    return dff

In [9]:
def get_plot(df_test, dict_nodes):
    shipper = df_test[['Shipper latitude', 'Shipper longitude']].iloc[0:1].rename({'Shipper latitude': 'Latitude', 'Shipper longitude': 'Longitude'}, axis = 1)
    receiver = df_test[['Receiver latitude', 'Receiver longitude']].rename({'Receiver latitude': 'Latitude', 'Receiver longitude': 'Longitude'}, axis = 1)
    routes = pd.concat([shipper, receiver]).reset_index()

    fig = go.Figure()
    for i in dict_nodes.keys():
        if routes.iloc[dict_nodes[i]].shape[0] > 2:
            fig.add_trace(go.Scattermapbox(lat=routes.iloc[dict_nodes[i]]['Latitude'], lon=routes.iloc[dict_nodes[i]]['Longitude'], mode = 'markers+lines', name = f'Route {i}'))
    fig.add_trace(go.Scattermapbox(lat=shipper['Latitude'], lon=shipper['Longitude'], mode = 'markers', marker = {'size': 12}, name = 'DC'))
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r": 0, "t": 40, "l": 0, "b": 40}, mapbox={'zoom': 6, "center": {'lat': routes['Latitude'].mean(), 'lon':  routes['Longitude'].mean()}})
    fig.show()

In [10]:
def plot_with_osrm(df_test, dict_nodes):
    shipper = df_test[['Shipper latitude', 'Shipper longitude']].iloc[0:1].rename({'Shipper latitude': 'Latitude', 'Shipper longitude': 'Longitude'}, axis = 1)
    receiver = df_test[['Receiver latitude', 'Receiver longitude']].rename({'Receiver latitude': 'Latitude', 'Receiver longitude': 'Longitude'}, axis = 1)
    routes = pd.concat([shipper, receiver]).reset_index()

    fig = go.Figure()
    for i in dict_nodes.keys():
        list_waypoints = get_waypoints_osrm(routes[['Longitude','Latitude']].iloc[dict_nodes[i]].values.tolist())['geometry']['coordinates']
        fig.add_trace(go.Scattermapbox(lat=[x[1] for x in list_waypoints], lon=[x[0] for x in list_waypoints], mode = 'lines', name = f'Route {i}'))
    fig.add_trace(go.Scattermapbox(lat=shipper['Latitude'], lon=shipper['Longitude'], mode = 'markers', marker = {'size': 12}, name = 'DC'))
    fig.add_trace(go.Scattermapbox(lat=receiver['Latitude'], lon=receiver['Longitude'], mode = 'markers', marker = {'size': 12}, name = 'Clients'))
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r": 0, "t": 40, "l": 0, "b": 40}, mapbox={'zoom': 6, "center": {'lat': routes['Latitude'].mean(), 'lon':  routes['Longitude'].mean()}})
    fig.show()

In [11]:
df = pd.read_csv(f'../data/poc3.csv', 
                 low_memory=False,
                 parse_dates=['Pickup date'], 
                 dtype={'DC zip': str, 'Zip': str}).sort_values(by='Pickup date')
df = df.dropna(subset = ['Distance (km)'])

df_evreux_day = df[(df['Distance (km)']<=200)&(df['DC name']=='DC Evreux')&(df['Pickup date']>=dt.datetime(2022,9,12))&(df['Pickup date']<=dt.datetime(2022,9,16))]
df_geolocations = df_evreux_day[['Client name','Receiver longitude', 'Receiver latitude','Shipper longitude', 'Shipper latitude']].drop_duplicates()

In [12]:
df_example = group_by_client_shipment(df_evreux_day).iloc[:100]

In [13]:
project_path = 's3://sfgdata/projects/sustainable-transport'
dataset_name = 'routing_example'
# Functions and main for VRP
df_test = pd.read_csv(f'{project_path}/processed/{dataset_name}.csv', 
                 low_memory=False)
df_test['Sender weight (kg)'] = np.ceil(df_test['Sender weight (kg)']).astype(int)

# VRP

In [14]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    dict_nodes = {}
    dict_distances = {}
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        nodes = []
        distance = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            nodes.append(node_index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            distance.append(routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id))
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        nodes.append(0)
        dict_nodes[vehicle_id] = nodes
        dict_distances[vehicle_id] = distance
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    return dict_distances, dict_nodes 

## Capacitated VRP

In [15]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = create_distance_matrix(df_test)
    data['demands'] = np.hstack((0, df_test['Sender weight (kg)'])).astype(int)
    capacity = 10000
    data['num_vehicles'] = round(df_test['Sender weight (kg)'].sum()/capacity)
    if data['num_vehicles'] < 3:
        data['num_vehicles'] = 3
    data['vehicle_capacities'] = np.full(shape=data['num_vehicles'],fill_value=capacity,dtype=int)
    data['depot'] = 0
    return data

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        dict_distances, dict_nodes = print_solution(data, manager, routing, solution)
        return dict_distances, dict_nodes

In [16]:
# Main
dict_distances, dict_nodes = main()

Objective: 1278
Route for vehicle 0:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 1:
 0 Load(0) ->  73 Load(7) ->  69 Load(20) ->  66 Load(146) ->  65 Load(157) ->  62 Load(159) ->  64 Load(161) ->  70 Load(164) ->  77 Load(165) ->  81 Load(170) ->  80 Load(176) ->  85 Load(261) ->  83 Load(265) ->  82 Load(267) ->  90 Load(352) ->  89 Load(353) ->  91 Load(355) ->  92 Load(359) ->  93 Load(364) ->  99 Load(367) ->  97 Load(627) ->  96 Load(7345) ->  78 Load(7969) ->  84 Load(7994) ->  86 Load(8013) ->  79 Load(8034) ->  76 Load(8035) ->  75 Load(8048) ->  74 Load(8051) ->  71 Load(8052) ->  0 Load(8052)
Distance of the route: 360m
Load of the route: 8052

Route for vehicle 2:
 0 Load(0) ->  95 Load(141) ->  100 Load(691) ->  98 Load(1791) ->  88 Load(1793) ->  87 Load(1797) ->  67 Load(1808) ->  44 Load(1813) ->  52 Load(1820) ->  55 Load(1892) ->  56 Load(1893) ->  57 Load(1963) ->  68 Load(1970) ->  60 Load(2364) ->  51 Load(2402) ->  50

In [12]:
pip install alns

     ---------------------------------------- 57.5/57.5 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [13]:
from alns import ALNS
from alns.accept import HillClimbing
from alns.select import RandomSelect
from alns.stop import MaxRuntime

import numpy.random as rnd

# Create ALNS and add one or more destroy and repair operators
alns = ALNS(rnd.RandomState(seed=42))
alns.add_destroy_operator(destroy)
alns.add_repair_operator(repair)

# Configure ALNS
select = RandomSelect(num_destroy=1, num_repair=1)  # see alns.select for others
accept = HillClimbing()  # see alns.accept for others
stop = MaxRuntime(60)  # 60 seconds; see alns.stop for others

# Run the ALNS algorithm
result = alns.iterate(init_sol, select, accept, stop)

# Retrieve the final solution
best = result.best_state
print(f"Best heuristic solution objective is {best.objective()}.")

NameError: name 'destroy' is not defined

#### Creating plots

In [17]:
a = 1
dict_routes = {}
for i in dict_nodes.keys():
    if len(dict_nodes[i])>2:
        dict_routes[a] = dict_nodes[i]
        a=a+1

In [18]:
get_plot(df_test, dict_routes)

In [19]:
plot_list(df_test, dict_routes)

NameError: name 'plot_list' is not defined

## Capacitated Vehicle Routing with distance capacity constraints

In [121]:
# Functions and main for VRP
df_test = df[df['Distance (km)']<=50].reset_index()
df_test['Sender weight (kg)'] = np.ceil(df_test['Sender weight (kg)']).astype(int)

In [122]:
df_test = df_test[(df_test['Pickup date'].dt.week == 2)&(df_test['Sender weight (kg)']<= 5000)]

C:\Users\SESA667490\AppData\Local\Temp\ipykernel_29128\1796854217.py:1: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.



In [156]:
df_test['Sender weight (kg)'].sum()/20

3003.85

In [155]:
df_test['Distance (km)'].max()

49.914

In [172]:
create_distance_matrix(df_test)*1000

array([[      0,   19000,   27000, ..., 1652000, 1661000, 1661000],
       [  19000,       0,   16000, ..., 1644000, 1653000, 1653000],
       [  27000,   16000,       0, ..., 1657000, 1667000, 1667000],
       ...,
       [1652000, 1644000, 1657000, ...,       0,    9000,    9000],
       [1661000, 1653000, 1667000, ...,    9000,       0,       0],
       [1661000, 1653000, 1667000, ...,    9000,       0,       0]])

In [10]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = create_distance_matrix(df_test)*1000
    data['demands'] = np.hstack((0, df_test['Sender weight (kg)'])).astype(int)
    capacity = 500000
    data['num_vehicles'] = 1000
    data['vehicle_capacities'] = np.full(shape=data['num_vehicles'],fill_value=capacity,dtype=int)
    data['depot'] = 0
    return data 

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        1000,  # no slack
        200000000,  # vehicle maximum travel distance in m
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        100,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    return solution, data

    # Print solution on console.
    if solution:
        dict_distances, dict_nodes = print_solution(data, manager, routing, solution)
        return dict_distances, dict_nodes
    else: 
        return 'No solution', 'No solution'

In [176]:
main()

(None,
 {'distance_matrix': array([[      0,   19000,   27000, ..., 1652000, 1661000, 1661000],
         [  19000,       0,   16000, ..., 1644000, 1653000, 1653000],
         [  27000,   16000,       0, ..., 1657000, 1667000, 1667000],
         ...,
         [1652000, 1644000, 1657000, ...,       0,    9000,    9000],
         [1661000, 1653000, 1667000, ...,    9000,       0,       0],
         [1661000, 1653000, 1667000, ...,    9000,       0,       0]]),
  'demands': array([   0,  238,   62,   25,   24,  126,    2,    4,   37,   24,    2,
           32,   12,    4,    8,    2, 3080,    1,    1,    1,    1,   12,
            4,    5,    7, 2168,  135,  277,  154,  779, 1518,  463,  453,
         1903,   54, 1439, 3531,   64,  149,  111,  184,   10,   53,   11,
           45,  240,  166,    1,    5,    2,    2,    2,    2,    7,    1,
            5,    1,    3,    1,    3,   15,    2,   15,   21,    2,    3,
            6,  388, 4343,   40,    1,   11,    1,    5,   67,  688,  157,
  

In [171]:
dict_distances

'No solution'

#### Creating plots

In [68]:
a = 1
dict_routes = {}
for i in dict_nodes.keys():
    if len(dict_nodes[i])>2:
        dict_routes[a] = dict_nodes[i]
        a=a+1

AttributeError: 'str' object has no attribute 'keys'

In [57]:
get_plot(df_test, dict_routes)

IndexError: positional indexers are out-of-bounds

In [58]:
plot_with_osrm(df_test, dict_routes)

IndexError: positional indexers are out-of-bounds

## Capacitated Vehicle Routing with Time Windows

In [50]:
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [51]:
time_matrix = [
                [0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
                [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
                [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
                [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
                [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
                [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
                [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
                [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
                [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
                [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
                [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
                [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
                [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
                [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
                [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
                [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
                [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0],
              ]

time_windows = [
                (0, 60),  # depot
                (7, 30),  # 1
                (10, 40),  # 2
                (16, 50),  # 3
                (10, 30),  # 4
                (0, 40),  # 5
                (5, 60),  # 6
                (0, 50),  # 7
                (5, 50),  # 8
                (0, 30),  # 9
                (10, 40),  # 10
                (10, 15),  # 11
                (0, 5),  # 12
                (5, 10),  # 13
                (7, 8),  # 14
                (10, 15),  # 15
                (11, 15),  # 16
              ]

demands = [0, 1, 1, 2, 4, 2, 4, 8, 3, 1, 2, 1, 2, 4, 4, 5, 5]

num_vehicles = 4
vehicle_capacities = [15, 15, 15, 15]

depot_index = 0

time_limit_seconds = 10 # time limit for calculation

In [52]:
def create_data_model(time_matrix, time_windows, num_vehicles, demands, vehicle_capacities, depot_index):
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = time_matrix
    data['time_windows'] = time_windows
    data['num_vehicles'] = num_vehicles
    data['demands'] = demands
    data['vehicle_capacities'] = vehicle_capacities
    data['depot'] = depot_index
    return data

In [53]:
"""Capacitated Vehicles Routing Problem (CVRP) with Time Windows."""

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    total_load = 0
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            time_var = time_dimension.CumulVar(index)
            route_load += data['demands'][node_index]
            plan_output += 'Place {0:>2} Arrive at {2:>2}min Depart at {3:>2}min (Load {1:>2})\n'.format(manager.IndexToNode(index), route_load, solution.Min(time_var), solution.Max(time_var))
            
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            
            
        time_var = time_dimension.CumulVar(index)
        total_time += solution.Min(time_var)
        plan_output +="Place {0:>2} Arrive at {2:>2}min \n\n".format(manager.IndexToNode(index), route_load, solution.Min(time_var), solution.Max(time_var))
        
        # route output
        plan_output += 'Load of the route: {}\n'.format(route_load)
        plan_output += 'Time of the route: {}min\n'.format(solution.Min(time_var))
        plan_output += "--------------------"
        
        print(plan_output)
        total_load += route_load

    print('Total load of all routes: {}'.format(total_load))
    print('Total time of all routes: {}min'.format(total_time))

    
def main():
    """Solve the VRP with time windows."""
    # Instantiate the data problem.
    data = create_data_model(time_matrix, time_windows, num_vehicles, demands, vehicle_capacities, depot_index)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')
    
    # Add Time Windows constraint.
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        30,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == 0:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

    # Add time window constraints for each vehicle start node.
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(data['time_windows'][0][0],
                                                data['time_windows'][0][1])

    # Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.time_limit.seconds = time_limit_seconds
    
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    return solution

In [5]:
solution = main()

Route for vehicle 0:
Place  0 Arrive at  0min Depart at  0min (Load  0)
Place  9 Arrive at  2min Depart at  5min (Load  1)
Place 14 Arrive at  7min Depart at  8min (Load  5)
Place 16 Arrive at 11min Depart at 11min (Load 10)
Place  0 Arrive at 18min 

Load of the route: 10
Time of the route: 18min
--------------------
Route for vehicle 1:
Place  0 Arrive at  0min Depart at  0min (Load  0)
Place 12 Arrive at  4min Depart at  5min (Load  2)
Place 11 Arrive at 10min Depart at 10min (Load  3)
Place 15 Arrive at 13min Depart at 13min (Load  8)
Place  3 Arrive at 19min Depart at 19min (Load 10)
Place  4 Arrive at 20min Depart at 20min (Load 14)
Place  1 Arrive at 22min Depart at 22min (Load 15)
Place  0 Arrive at 28min 

Load of the route: 15
Time of the route: 28min
--------------------
Route for vehicle 2:
Place  0 Arrive at  0min Depart at  0min (Load  0)
Place  7 Arrive at  2min Depart at  2min (Load  8)
Place 13 Arrive at  5min Depart at  5min (Load 12)
Place  0 Arrive at  9min 

Load o